In [1]:
"""Testing out Basket Performance using JPMaQS."""

'Testing out Basket Performance using JPMaQS.'

In [2]:
import yaml
from macrosynergy.management.dq import DataQueryInterface
from macrosynergy.panel.basket_performance import *
import warnings

warnings.filterwarnings("ignore")

In [3]:
def dq_output(tickers=None, xcats=None, cids=None, metrics=['value'],
              start_date='2000-01-01'):

    with open("config.yml", 'r') as f:
        cf = yaml.load(f, Loader=yaml.FullLoader)

    dq = DataQueryInterface(username=cf["dq"]["username"], password=cf["dq"]["password"],
                            crt="api_macrosynergy_com.crt",
                            key="api_macrosynergy_com.key")

    df_ts = dq.download(tickers=tickers, xcats=xcats, cids=cids, metrics=metrics,
                        start_date=start_date)

    return df_ts

In [4]:
cids_dmca = ['AUD', 'CAD', 'CHF', 'EUR', 'GBP', 'JPY', 'NOK', 'NZD', 'SEK',
                 'USD']  # DM currency areas
cids_dmec = ['DEM', 'ESP', 'FRF', 'ITL', 'NLG']  # DM euro area countries
cids_latm = ['ARS', 'BRL', 'COP', 'CLP', 'MXN', 'PEN']  # Latam countries
cids_emea = ['HUF', 'ILS', 'PLN', 'RON', 'RUB', 'TRY', 'ZAR']  # EMEA countries
cids_emas = ['CNY', 'HKD', 'IDR', 'INR', 'KRW', 'MYR', 'PHP', 'SGD', 'THB',
                 'TWD']  # EM Asia countries

cids_eufx = ['CHF', 'HUF', 'NOK', 'PLN', 'RON', 'SEK']  # EUR benchmark
cids_g2fx = ['GBP', 'RUB', 'TRY']  # dual benchmark
cids_usfx = ['AUD', 'BRL', 'CAD', 'CLP', 'CNY', 'COP', 'EUR', 'IDR', 'ILS', 'INR',
             'JPY', 'KRW', 'MYR',
             'MXN', 'NZD', 'PEN', 'PHP', 'SGD', 'THB', 'TWD', 'ZAR']

In [5]:
cids = ['AUD', 'GBP', 'CHF', 'CAD']
metrics = ['value']

rets = ['EQXR_NSA', 'FXXR_NSA']
cry = ['FXCRY_NSA', 'EQCRY_NSA']

xcats = rets + cry

dfd = dq_output(cids=cids, xcats=xcats, metrics=metrics, start_date="2000-01-01")
print(dfd)

# Todo: add value category with only positive values that can serve as weight
black = {'AUD': ['2000-01-01', '2003-12-31'], 'GBP': ['2018-01-01', '2100-01-01']}
contracts = ['AUD_FX', 'AUD_EQ', 'CHF_FX', 'GBP_EQ', 'CAD_EQ']

       cid       xcat  real_date  value
0      AUD  EQCRY_NSA 2000-01-03    NaN
1      AUD  EQCRY_NSA 2000-01-04    NaN
2      AUD  EQCRY_NSA 2000-01-05    NaN
3      AUD  EQCRY_NSA 2000-01-06    NaN
4      AUD  EQCRY_NSA 2000-01-07    NaN
...    ...        ...        ...    ...
91355  GBP   FXXR_NSA 2021-11-15    NaN
91356  GBP   FXXR_NSA 2021-11-16    NaN
91357  GBP   FXXR_NSA 2021-11-17    NaN
91358  GBP   FXXR_NSA 2021-11-18    NaN
91359  GBP   FXXR_NSA 2021-11-19    NaN

[91360 rows x 4 columns]


In [6]:
gdp_figures = [17.0, 17.0, 41.0, 9.0, 250]
dfd_2 = basket_performance(dfd, contracts, ret='XR_NSA', cry=None, blacklist=black,
                           weight_meth='fixed', weights=gdp_figures,
                           wgt=None, max_weight=0.35, return_weights=False, 
                           data_query=True)

print(dfd_2)

dfd_3 = basket_performance(dfd, contracts, ret='XR_NSA', cry='CRY_NSA', blacklist=black,
                           weight_meth='fixed', weights=gdp_figures,
                           wgt=None, max_weight=0.35, return_weights=False, 
                           data_query=True)

print(dfd_3)

dfd_4 = basket_performance(dfd, contracts, ret='XR_NSA', cry=None,
                           weight_meth='invsd', weights=None,
                           lback_meth="xma", lback_periods=21,
                           wgt=None, max_weight=0.4, return_weights=True, 
                           data_query=True)
print(dfd_4)

dfd_5 = basket_performance(dfd, contracts, ret='XR_NSA', cry=None,
                           weight_meth='equal', wgt=None, max_weight=0.41,
                           return_weights=False, data_query=True)
print(dfd_5)

# The only weighting method which has not been tested is weight_meth = "values".
# Previously, the categories defined were: xcats = ['FX_XR', 'FX_CRY', 'EQ_XR', 'EQ_CRY', 'FX_WGT', 'EQ_WGT'] where
# ['FX_WGT', 'EQ_WGT'] are contrived categories designed to act as weights on the other categories in the List: FX & EQ.
# The code reflects the above input format. However, the current logic, I believe, is insufficient.

# Firstly, in order to support mutiple contracts on a single cross-section, a corresponding number of external weight
# categories are required but, in the current format, the parameter wgt receives a string which only works if the aforementioned
# weight categories, in the dataframe dfd, have the same symbol. 
# For instance, "WGT". What would be the DataQuery equivalent ? The code being referenced is: 
# ticks_wgt = [c + "_" + wgt for c in contracts]
# dfw_ticks_wgt = dfx[dfx["ticker"].isin(ticks_wgt)]

contracts = ['AUD_FX', 'AUD_EQ', 'CHF_FX', 'GBP_EQ', 'CAD_EQ']
xcats = ['FX_XR', 'FX_CRY', 'EQ_XR', 'EQ_CRY', 'FX_WGT', 'EQ_WGT']
# Secondly, the categories and contracts, defined above and used for testing in basket_performance.py file, present 
# a perfect balance. There are two unique categories represented in the List of contracts, and consequently 
# leaving two external categories, ['FX_WGT', 'EQ_WGT'], in the dataframe, dfd, that can be used as the weights.
# It would seem unrealistic for all user engagements to also possess such a paradigm. What if the dataframe, dfd, is only
# defined over three categories and two referenced in the contract List where one cross-section is defined over both
# categories ? There would be an imbalance in the current setup.

# Maybe I've misunderstood or I'm overthinking but thought I'd present my concerns.

              ticker  real_date     value
0     GLB_ALL_XR_NSA 2000-01-03  0.113118
1     GLB_ALL_XR_NSA 2000-01-04 -1.797716
2     GLB_ALL_XR_NSA 2000-01-05 -1.026415
3     GLB_ALL_XR_NSA 2000-01-06 -0.744445
4     GLB_ALL_XR_NSA 2000-01-07  1.378595
...              ...        ...       ...
5705  GLB_ALL_XR_NSA 2021-11-15 -0.005907
5706  GLB_ALL_XR_NSA 2021-11-16 -0.300870
5707  GLB_ALL_XR_NSA 2021-11-17 -0.405056
5708  GLB_ALL_XR_NSA 2021-11-18  0.004191
5709  GLB_ALL_XR_NSA 2021-11-19  0.023986

[5710 rows x 3 columns]
                ticker  real_date     value
0       GLB_ALL_XR_NSA 2000-01-03  0.113118
1       GLB_ALL_XR_NSA 2000-01-04 -1.797716
2       GLB_ALL_XR_NSA 2000-01-05 -1.026415
3       GLB_ALL_XR_NSA 2000-01-06 -0.744445
4       GLB_ALL_XR_NSA 2000-01-07  1.378595
...                ...        ...       ...
11415  GLB_ALL_CRY_NSA 2021-11-15  0.000000
11416  GLB_ALL_CRY_NSA 2021-11-16  0.000000
11417  GLB_ALL_CRY_NSA 2021-11-17  0.000000
11418  GLB_ALL_CRY_NSA 2021-11-